# Sire-EMLE

The `sire` QM/MM implementation takes advantage of the new means of writing [platform independent force calculations](http://docs.openmm.org/development/developerguide/09_customcppforceimpl.html) introduced in [OpenMM](http://openmm.org/) 8.1. This allows us to interface with any external package to modify atomic forces within the OpenMM context. While OpenMM already directly supports ML/MM simulations via the [OpenMM-ML](https://github.com/openmm/openmm-ml) package, it is currently limited to specific backends and only supports mechanical embedding. The `sire` QM/MM implementation performs the QM calculation using the [emle-engine](https://github.com/chemle/emle-engine) package, which has support for a wide range of backends and embedding models, importantly providing a simple and efficient ML model for electrostatic embedding.

Here are some useful links:

* [Paper](https://doi.org/10.26434/chemrxiv-2022-rknwt-v3) on the original EMLE methodology.
* [emle-engine](https://github.com/chemle/emle-engine) GitHub repository.
* [Preprint](https://doi.org/10.26434/chemrxiv-2023-6rng3-v2) on alanine-dipeptide conformational landscape study.
* Sire-EMLE [tutorials](https://github.com/OpenBioSim/sire/blob/feature_emle/doc/source/tutorial/partXX).

In order to use QM/MM functionality within `sire` you will first need to create the following `conda` environment:

```bash
$ git clone https://github.com/chemle/emle-engine.git
$ cd emle-engine
$ conda env create -f environment_sire.yaml
$ conda activate emle-sire
$ pip install -e .
```

In this tutorial, we will perform a short ML/MM simulation of alanine dipeptide in water. First, let us load the molecular system:

In [ ]:
import sire as sr
mols = sr.load_test_files("ala.crd", "ala.top")

## Creating an EMLE calculator

Next we will create an `emle-engine` calculator to perform the QM (or ML) calculation for the dipeptide along with the ML electrostatic embedding. Since this is a small molecule it isn't beneficial to perform the calculation on a GPU, so we will use the CPU instead.

In [ ]:
from emle.calculator import EMLECalculator
calculator = EMLECalculator(device="cpu")

By default, `emle-engine` will use [TorchANI](https://aiqm.github.io/torchani/) as the backend for in vacuo calculation of energies and gradients using the ANI-2x model. However, it is possible to use a wide variety of other backends, including your own as long as it supports the standand [Atomic Simulation Environment (ASE) calculator interface](https://wiki.fysik.dtu.dk/ase/). For details, see the [backends](https://github.com/chemle/emle-engine#backends) section of the emle-engine documentation. At present, the default embedding model provided with emle-engine supports only the elements H, C, N, O, and S. We plan on adding support for other elements in the near future.

## Creating a QM engine

We now need to set up the molecular system for the QM/MM simulation and create an engine to perform the calculation:

In [ ]:
qm_mols, engine = sr.qm.emle(mols, mols[0], calculator, "7.5A", 20)

Here the first argument is the molecules that we are simulating, the second selection coresponding to the QM region (here this is the first molecule), and the third is calculator that was created above. The fourth and fifth arguments are optional, and specify the QM cutoff distance and the neigbour list update frequency respectively. (Shown are the default values.) The function returns a modified version of the molecules containing a "merged" dipeptide that can be interpolated between MM and QM levels of theory, along with an engine. The engine registers a Python callback that uses `emle-engine` to perform the QM calculation.

The selection syntax for QM atoms is extremely flexible. Any valid search string, atom index, list of atom indicies, or molecule view/container that can be used. Support for modelling partial molecules at the QM level is provided via the link atom approach, via the charge shifting method. For details of this implementation, see, e.g., the NAMD user guide [here](https://www.ks.uiuc.edu/Research/qmmm/). While we support multiple QM fragments, we do not currently support multiple independent QM regions. We plan on adding support for this in the near future.

## Running a QM/MM simulation

Next we need to create a dynamics object to perform the simulation. For QM/MM simulations it is recommended to use a 1 femtosecond timestep and no constraints. In this example we will use the `lambda_interpolate` keyword to interpolate the dipeptide potential between pure MM (λ=0) and QM (λ=1) over the course of the simulation, which can be used for end-state correction of binding free energy calculations.

In [ ]:
d = qm_mols.dynamics(
    timestep="1fs",
    constraint="none",
    qm_engine=engine,
    lambda_interpolate=[0, 1],
    platform="cpu",
)

We can now run the simulation. The options below specify the run time, the frequency at which trajectory frames are saved, and the frequency at which energies are recorded. The energy_frequency also specifies the frequency at which the λ value is updated.

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1"
d.run("0.1ps", frame_frequency="0.01ps", energy_frequency="0.01ps")

<div class="alert alert-block alert-info">
⚠️ Updating λ requires the updating of force field parameters in the OpenMM context. For large systems, this can be quite slow so it isn't recommended to set the energy_frequency to a value that is too small. We have a custom <a href=https://github.com/chryswoods/openmm">fork</a> of OpenMM that provides a significant speedup for this operation by only updating a subset of the parameters. Installation instructions can be provided on request.
</div>

<div class="alert alert-block alert-info">
⚠️ If you don't require a trajectory file, then better performance can be achieved leaving the frame_frequency keyword argument unset.
</div>

<div class="alert alert-block alert-info">
⚠️ Currently requires the use of <a href="https://lab-cosmo.github.io/librascal/#/">librascal</a> for the calculation of SOAP (Smooth Overlap of Atomic Positions) descriptors. This is a serial code, so you may see better performance by restricting the number of OpenMP threads to 1, e.g. by setting the OMP_NUM_THREADS environment variable.
</div>

Once the simulation has finished we can get back the trajectory of energy values. This can be obtained as a [pandas](https://pandas.pydata.org/) `DataFrame`, allowing for easy plotting and analysis. The table below shows the instantaneous kintetic and potential energies as a function of λ, along with the pure MM and QM potential energies. (Times are in picoseconds and energies are in kcal/mol.)

In [ ]:
d.energy_trajectory(to_pandas=True)

## Interfacing with OpenMM-ML

In the example above we used a sire dynamics object d to run the simulation. This is wrapper around a standard `OpenMM` context object, providing a simple convenience functions to make it easier to run and analyse simulations. (It is easy to extract the system and forces from the context in order to create a customised simulation of your own.) However, if you are already familiar with OpenMM, then it is possible to use emle-engine with OpenMM directly. This allows for fully customised simulations, or the use of [OpenMM-ML](https://github.com/openmm/openmm-ml) as the backend for calculation of the intramolecular force for the QM region.

To use `OpenMM-ML` as the backend for the QM calculation, you will first need to install the package:

```bash
$ conda install -c conda-forge openmm-ml
```

Next, you will need to create an `MLPotential` for desired backend. Here we will use ANI-2x, as was used for the EMLECalculator above. The

In [ ]:
import openmm
from openmmml import MLPotential
potential = MLPotential("ani2x")

Since we are now using the `MLPotential` for the QM calculation, we need to create a new `EMLECalculator` object with no backend, i.e. one that only computes the electrostatic embedding:

In [ ]:
calculator = EMLECalculator(backend=None, device="cpu")

Next we create a new engine bound to the calculator:

In [ ]:
qm_mols, engine = sr.qm.emle(mols, mols[0], calculator)

Rather than using this engine with a `sire` dynamics object, we can instead extract the underlying `OpenMM` force object and add it to an existing `OpenMM` system. The forces can be extracted from the engine as follows:

In [ ]:
emle_force, interpolation_force = engine.get_forces()

The `emle_force` object is the `OpenMM` force object that calculates the electrostatic embedding interaction. The `interpolation_force` is a null `CustomBondForce` object that contains a `lambda_emle` global parameter than can be used to scale the electrostatic embedding interaction. (By default, this is set to 1, but can be set to any value between 0 and 1.)

<div class="alert alert-block alert-info">
⚠️ The interpolation_force has no energy contribution. It is only required as there is currently no way to add global parameters to the EMLEForce.
</div>

Since we want to use electrostatic embedding, we will also need to zero the charges on the atoms within the QM region before creating an `OpenMM` system. If not, then we would also calculate the mechanical embedding interaction. This can be done using the `qm_mols` object generated above. This system is *perturbable* so can be converted between an MM reference state and QM perturbed state. Here we require the perturbed state, which has zeroed charges for the QM region:

In [ ]:
qm_mol = sr.morph.link_to_perturbed(qm_mols[0])
qm_mols.update(qm_mol)
qm_mol.property("charge")

We now write the modified system to an AMBER format topology and coordinate file so that we can load them with `OpenMM`:

In [ ]:
sr.save(qm_mols, "ala_qm", ["prm7", "rst7"])

We can now read them back in with OpenMM:

In [ ]:
prmtop = openmm.app.AmberPrmtopFile("ala_qm.prm7")
inpcrd = openmm.app.AmberInpcrdFile("ala_qm.rst7")

Next we use the prmtop to create the MM system:

In [ ]:
mm_system = prmtop.createSystem(
    nonbondedMethod=openmm.app.PME,
    nonbondedCutoff=7.5 * openmm.unit.angstrom,
    constraints=openmm.app.HBonds
)

In oder to create the ML system, we first define the ML region. This is a list of atom indices that are to be treated with the ML model.

In [ ]:
ml_atoms = list(range(qm_mols[0].num_atoms()))

We can now create the ML system:

In [ ]:
ml_system = potential.createMixedSystem(prmtop.topology, mm_system, ml_atoms, interpolate=True)

By setting `interpolate=True` we are telling the `MLPotential` to create a mixed system that can be interpolated between MM and ML levels of theory using the `lambda_interpolate` global parameter. (By default this is set to 1.)

<div class="alert alert-block alert-info">
⚠️ If you choose not to add the emle interpolation force to the system, then the EMLEForce will also use the lambda_interpolate global parameter. This allows for the electrostatic embedding to be alongside or independent of the ML model.
</div>

We can now add the `emle` forces to the system:

In [ ]:
ml_system.addForce(emle_force)
ml_system.addForce(interpolation_force)

In order to run a simulation we need to create an integrator and context. First we create the integrator:

In [ ]:
integrator = openmm.LangevinMiddleIntegrator(
    300 * openmm.unit.kelvin,
    1.0 / openmm.unit.picosecond,
    0.002 * openmm.unit.picosecond
)

And finally the context:

In [ ]:
context = openmm.Context(ml_system, integrator)
context.setPositions(inpcrd.positions)

Let's check the global parameters:

In [ ]:
for param in context.getParameters():
    print(param, context.getParameter(param))